In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
import numpy as np

def StressTransform3D(Pf, SH, Sh, Sv, phi, theta):
    # construct stress tensor field: shape (...,3,3)
    s = np.zeros(Pf.shape + (3,3))
    s[...,0,0] = SH - Pf
    s[...,1,1] = Sh - Pf
    s[...,2,2] = Sv - Pf
    
    # pre-calculate trig
    cos_phi, sin_phi = np.cos(np.radians(phi)), np.sin(np.radians(phi))
    cos_theta, sin_theta = np.cos(np.radians(theta)), np.sin(np.radians(theta))
    
    # rotation matrices
    Rz = np.array([[cos_phi,-sin_phi,0],
                   [sin_phi, cos_phi,0],
                   [0,0,1]])
    Rx = np.array([[1,0,0],
                   [0,cos_theta,-sin_theta],
                   [0,sin_theta, cos_theta]])
    R = Rx @ Rz
    
    # rotate stress tensor: R s R^T
    # einsum does the batched matrix multiplication
    result = np.einsum("ab,...bc,cd->...ad", R, s, R.T)
    
    # extract stresses
    tau1 = result[...,2,0]
    tau2 = result[...,2,1]
    tau  = np.sqrt(tau1**2 + tau2**2)
    sigma = result[...,2,2]
    
    return sigma, tau


In [15]:
# input stress state
SH_grad = 15.25 #maximum horizontal stress gradient in MPa/km
Sh_grad = 4.89 #minimun horizontal stress gradient in MPa/km
Sv_grad = 11.69#vertical stress gradient in MPa/km
SH_azi = 292.13 #maximum horizontal stress direction in degree, relative to north clockwise, range [0,180]
mu = 0.6 #coefficient of friction
cohesion = 1 # fault cohesion in MPa

fault_strike = 10
fault_dip = 90

# load data
coor_fault = np.load('results/JD_Sula_2025_flow_coor&fault.npy')
pres = np.load('results/case1_PRES.npy')

x_coor = coor_fault[:,:,:,0]
y_coor = coor_fault[:,:,:,1]
z_coor = coor_fault[:,:,:,2]

pres_slice = pres[:,:,:,2]/1000 # convert to MPa

SH_stress = z_coor /1000 * SH_grad
Sh_stress = z_coor /1000 * Sh_grad
Sv_stress = z_coor /1000 * Sv_grad

# compute rotation angles
phi = SH_azi - fault_strike
theta = fault_dip

sigma, tau = StressTransform3D(pres_slice, SH_stress, Sh_stress, Sv_stress, phi, theta)

fault_slip = ((tau - cohesion) / sigma >= mu).astype(int)
np.save('results/fault_slip.npy', fault_slip)

In [12]:
fault_slip.shape

(107, 117, 79)

In [13]:
print(np.min(fault_slip))
print(np.max(fault_slip))

0
1
